## EOS_RAM

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
max_ram_size = 64 * 1024 * 1024 * 1024 # 64G RAM
total_ram_bytes_reserved = 0 # 已占用的RAM
total_ram_size_free = max_ram_size-total_ram_bytes_reserved # RAM余额
system_token_supply = 1000000000 # EOS发行量

In [ ]:
supply = {'amount':100000000000000, 'symbol':'RAMCORE'} 
base = {'amount':total_ram_size_free, 'symbol':'RAM', 'weight':.5}
quote = {'amount':system_token_supply / 1000, 'symbol':'EOS', 'weight':.5}

In [ ]:
def convert_to_exchange(c, in_token):
    global supply
    R = supply['amount'] 
    C = c['amount'] + in_token['amount']
    F = c['weight'] / 1000.0
    T = in_token['amount']
    ONE = 1.0
    E = -R * (ONE - math.pow( ONE + T / C, F))
    #P = C/(F*R)
    supply['amount'] += E;
    c['amount'] += in_token['amount'];
    return c, {'amount': E,'symbol': supply['symbol']}

In [ ]:
def convert_from_exchange(c, in_token):
    global supply
    R = supply['amount'] - in_token['amount']
    C = c['amount']
    F = 1000.0 / c['weight']
    E = in_token['amount']
    ONE = 1.0
    T = C * (math.pow(ONE + E/R, F) - ONE)
    supply['amount'] -= in_token['amount']
    c['amount'] -= T;
    return c, {'amount': T,'symbol': c['symbol']}

In [ ]:
def convert(from_token, to_symbol):
    global supply
    global base
    global quote
    sell_symbol = from_token['symbol']
    ex_symbol = supply['symbol']
    base_symbol = base['symbol']
    quote_symbol = quote['symbol']
    if sell_symbol != ex_symbol:
        if sell_symbol == base_symbol:
            base, from_token = convert_to_exchange(base, from_token)
        elif sell_symbol == quote_symbol:
            quote, from_token = convert_to_exchange(quote, from_token)
        else:
            print("invalid sell")
    else:
        if to_symbol == base_symbol:
            base, from_token = convert_from_exchange(base, from_token )
        elif to_symbol == quote_symbol:
            quote, from_token = convert_from_exchange(quote, from_token )
        else:
            print("invalid conversion")
    
    if to_symbol != from_token['symbol']:
        return convert(from_token, to_symbol)
    return from_token

In [ ]:
def buy_ram(amount):
    fee = amount / 200
    amount_after_fee = amount - fee
    ram = convert({'symbol':'EOS','amount':amount_after_fee},'RAM')
    price = amount / (ram['amount'] / 1024.0)
    percent = 1 - base['amount'] / max_ram_size
    return price, ram['amount'], percent

In [ ]:
def buy_ram_bytes(amount):
    eos = convert({'symbol':'RAM','amount':amount},'EOS')
    price, ram, percent = buy_ram(eos['amount'])
    return price, ram, percent, eos['amount']

In [ ]:
def sell_ram(amount):
    eos = convert({'symbol':'RAM','amount':amount},'EOS')
    fee = eos['amount'] / 200
    eos_after_fee = eos['amount'] - fee
    price = eos['amount'] / ( amount / 1024.0)
    percent = 1 - base['amount'] / max_ram_size
    return price, eos_after_fee, percent

In [ ]:
def restart():
    global supply
    global base
    global quote
    supply = {'amount':100000000000000, 'symbol':'RAMCORE'} #
    base = {'amount':total_ram_size_free, 'symbol':'RAM', 'weight':.5}
    quote = {'amount':system_token_supply / 1000, 'symbol':'EOS', 'weight':.5}

In [ ]:
def add_ram(amount):
    # bytes
    global base
    base['amount'] += amount

In [ ]:
wanner_percent = 0.8
x = []
y = []
#z = []
step = 1000
while ((1 - base['amount'] / max_ram_size) <= wanner_percent):
    price, ram, percent = buy_ram(step)
    x.append(percent)
    y.append(price)
    #z.append(ram)
plt.figure(figsize=(8,4))
plt.plot(np.array(x),np.array(y),label="$price$",color="red",linewidth=2)
plt.xlabel("Ram Sold Percentage")
plt.ylabel("Price")
plt.title("EOS RAM")
plt.ylim(0.0,max(y)*1.1)
plt.legend()
plt.show()

In [ ]:
price, ram, percent = buy_ram(1)
print(price, ram, percent)
print(quote)